## Github repository
This notebook and related resources are published on the git public repository: https://github.com/DSwing/IML

## Source
The data source is on Kaggle at https://www.kaggle.com/datasets/jainaru/thyroid-disease-data/data  
Introductory Paper published on European Archives of Oto-Rhino-Laryngology:   
Machine learning for risk stratification of thyroid cancer patients: a 15-year cohort study
By Shiva Borzooei, Giovanni Briganti, Mitra Golparian, Jerome R. Lechien, Aidin Tarokhian. 2023

## Data Description
A cohort of 383 patients was observed for a minimum duration of 10 years within a 15-year timeframe. Thirteen clinicopathologic features were assessed to predict recurrence potential.
It's a tabular dataset with 383 rows x 17 columns.  

List of features
1. Age: The age of the patient at the time of diagnosis or treatment.
2. Gender: The gender of the patient (male or female).
3. Smoking: Whether the patient is a smoker or not.
4. Hx Smoking: Smoking history of the patient (e.g., whether they have ever smoked).
5. Hx Radiotherapy: History of radiotherapy treatment for any condition.
6. Thyroid Function: The status of thyroid function, possibly indicating if there are any abnormalities.
7. Physical Examination: Findings from a physical examination of the patient, which may include palpation of the thyroid gland and surrounding structures.
8. Adenopathy: Presence or absence of enlarged lymph nodes (adenopathy) in the neck region.
9. Pathology: Specific types of thyroid cancer as determined by pathology examination of biopsy samples.
10. Focality: Whether the cancer is unifocal (limited to one location) or multifocal (present in multiple locations).
11. Risk: The risk category of the cancer based on various factors, such as tumor size, extent of spread, and histological type.
12. T: Tumor classification based on its size and extent of invasion into nearby structures.
13. N: Nodal classification indicating the involvement of lymph nodes.
14. M: Metastasis classification indicating the presence or absence of distant metastases.
15. Stage: The overall stage of the cancer, typically determined by combining T, N, and M classifications.
16. Response: Response to treatment, indicating whether the cancer responded positively, negatively, or remained stable after treatment.
17. Recurred: Indicates whether the cancer has recurred after initial treatment.

## Task
The goal is to provide a model for prediction of the recurred cases after the initial treatment.  
Three different classifiers will be used (KNN, Decision Tree, Random Forrest).  
I will compare the accuracy on train data for different values of hyperparameters and take the best model in each group. 
Then I will keep the best model that results in the best accuracy on the test dataset and best AUC value.

In [18]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, roc_auc_score
import joblib

The joblib library is used to export the trained model in pkl format to use it in an application for prection. A little flask application has been created to make predictions based on the trained model, with an html interface. 

In [19]:
data = pd.read_csv('./sample_data/Thyroid_Diff.csv')

## EDA

In [20]:
data.head()

,Age,Gender,Smoking,Hx Smoking,Hx Radiothreapy,Thyroid Function,Physical Examination,Adenopathy,Pathology,Focality,Risk,T,N,M,Stage,Response,Recurred
0,27,F,No,No,No,Euthyroid,Single nodular goiter-left,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Indeterminate,No
1,34,F,No,Yes,No,Euthyroid,Multinodular goiter,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Excellent,No
2,30,F,No,No,No,Euthyroid,Single nodular goiter-right,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Excellent,No
3,62,F,No,No,No,Euthyroid,Single nodular goiter-right,No,Micropapillary,Uni-Focal,Low,T1a,N0,M0,I,Excellent,No
4,62,F,No,No,No,Euthyroid,Multinodular goiter,No,Micropapillary,Multi-Focal,Low,T1a,N0,M0,I,Excellent,No


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383 entries, 0 to 382
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Age                   383 non-null    int64 
 1   Gender                383 non-null    object
 2   Smoking               383 non-null    object
 3   Hx Smoking            383 non-null    object
 4   Hx Radiothreapy       383 non-null    object
 5   Thyroid Function      383 non-null    object
 6   Physical Examination  383 non-null    object
 7   Adenopathy            383 non-null    object
 8   Pathology             383 non-null    object
 9   Focality              383 non-null    object
 10  Risk                  383 non-null    object
 11  T                     383 non-null    object
 12  N                     383 non-null    object
 13  M                     383 non-null    object
 14  Stage                 383 non-null    object
 15  Response              383 non-null    ob

## Cleaning  
There are no null values in the dataset. Looking for duplication.

In [22]:
data[data.duplicated()].shape

(19, 17)

19 rows are duplicated, will be removed.

In [23]:
data.drop_duplicates(inplace=True)

Looking at the summary of quantitative variables.

In [24]:
data.describe()

,Age
count,364.00000
mean,41.25000
std,15.31436
min,15.00000
25%,30.00000
50%,38.00000
75%,52.00000
max,82.00000


and a summary of the nominal variables

In [25]:
for column in data.columns[1:]:
    print(data[column].value_counts())
    print(10*'*')

Gender
F    293
M     71
Name: count, dtype: int64
**********
Smoking
No     315
Yes     49
Name: count, dtype: int64
**********
Hx Smoking
No     336
Yes     28
Name: count, dtype: int64
**********
Hx Radiothreapy
No     357
Yes      7
Name: count, dtype: int64
**********
Thyroid Function
Euthyroid                      313
Clinical Hyperthyroidism        20
Subclinical Hypothyroidism      14
Clinical Hypothyroidism         12
Subclinical Hyperthyroidism      5
Name: count, dtype: int64
**********
Physical Examination
Multinodular goiter            135
Single nodular goiter-right    127
Single nodular goiter-left      88
Normal                           7
Diffuse goiter                   7
Name: count, dtype: int64
**********
Adenopathy
No           258
Right         48
Bilateral     32
Left          17
Extensive      7
Posterior      2
Name: count, dtype: int64
**********
Pathology
Papillary         271
Micropapillary     45
Follicular         28
Hurthel cell       20
Name: count, dty

A natural order is recognizable in some variables.

In [26]:
data.Risk = pd.Categorical(data.Risk, categories=['Low', 'Intermediate', 'High'], ordered=True)
data.Stage = pd.Categorical(data.Stage, categories=['I', 'II', 'III', 'IVA', 'IVB'], ordered=True)
data['T'] = pd.Categorical(data['T'], categories=['T1a', 'T1b', 'T2', 'T3a', 'T3b', 'T4a', 'T4b'], ordered=True)

## Visualizations  
Plots of single variables distributions

In [27]:

alt.Chart(data).mark_bar().encode(
    alt.X(alt.repeat("column")),
    alt.Y(aggregate='count')
).transform_bin(
    'Age', field='Age',  bin=alt.Bin(maxbins=5)
).properties(
    width=100,
    height=100
).repeat(
    column=list(data.columns),
)


alt.RepeatChart(...)

Plots of two variables distributions.

In [28]:

alt.Chart(data).mark_rect().encode(
    alt.X(alt.repeat("column")),
    alt.Y(alt.repeat("row")),
    alt.Color(aggregate='count', scale=alt.Scale(type='log'))
).transform_bin(
    'Age', field='Age',  bin=alt.Bin(maxbins=5)
).properties(
    width=100,
    height=100
).repeat(
    row=list(data.columns),
    column=list(data.columns)
)


alt.RepeatChart(...)

These plots show that some correlation is present among the variables. Some collinearity issue can be present.  
To get more into the problem, we need to do some preprocessing to get the correlation matrix.

## Preprocessing   
We need to transform all categorical values to numeric.

In [29]:
# Age normalization
data[['Age']] = MinMaxScaler().fit_transform(data[['Age']])

# Categorical features that will be label encoded.
# Label encoding converts each unique category to a different integer value.
# This method is suitable for ordinal categorical features where the order matters.
label_method=['Gender', 'Hx Radiothreapy', 'Smoking', 'Hx Smoking', 'Focality', 'Risk', 'T', 'N', 'M', 'Stage', 'Recurred']
for column in label_method:
  label_encoder = LabelEncoder()
  data[column] = label_encoder.fit_transform(data[column])

# Categorical features that will be one-hot encoded.
# One-hot encoding creates binary columns for each category in the original feature.
# This method is suitable for nominal categorical features where the order does not matter.
onehot_method = ['Thyroid Function', 'Physical Examination', 'Adenopathy', 'Pathology', 'Response']
for column in onehot_method:
    onehot_encoder = OneHotEncoder(sparse_output=False, drop='first')
    encoded = onehot_encoder.fit_transform(data[[column]])
    column_names = onehot_encoder.get_feature_names_out([column])
    data_encoded = pd.DataFrame(encoded, columns=column_names, index=data.index)
    data = pd.concat([data, data_encoded], axis=1)
    data.drop(columns=[column], inplace=True)

This step ensures that all categorical data is converted into a numerical format that machine learning algorithms can interpret.  
Now we can get the design matrix and the response

In [30]:
X, y = data.drop(columns=['Recurred']), data.loc[:,'Recurred']

In [31]:
X.head()

,Age,Gender,Smoking,Hx Smoking,Hx Radiothreapy,Focality,Risk,T,N,M,...,Adenopathy_Left,Adenopathy_No,Adenopathy_Posterior,Adenopathy_Right,Pathology_Hurthel cell,Pathology_Micropapillary,Pathology_Papillary,Response_Excellent,Response_Indeterminate,Response_Structural Incomplete
0,0.179104,0,0,0,0,1,2,0,0,0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.283582,0,0,1,0,1,2,0,0,0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.223881,0,0,0,0,1,2,0,0,0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.701493,0,0,0,0,1,2,0,0,0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.701493,0,0,0,0,0,2,0,0,0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


Now we can calculate and plot the correlation matrix

In [32]:
correlation_matrix = data.corr()
correlation_matrix_long = correlation_matrix.reset_index().melt(id_vars='index')
correlation_matrix_long.columns = ['V1','V2','Correlation']
alt.Chart(correlation_matrix_long).mark_rect().encode(
    alt.X('V1:O', axis=alt.Axis(labelAngle=90)),
    alt.Y('V2:O'),
    alt.Color('Correlation:O', scale=alt.Scale(scheme='redblue',domainMid=0)),
    tooltip=['V1','V2','Correlation']
).properties(title='Correlation Matrix')

alt.Chart(...)

There are some values that are highly correlated: we know that the Stage variable is usually calculated from M, N and T values, so I will remove M,N,T and keep only the Stage. I will also remove the predictors that have a very low correlation with the Recurred variable.

In [33]:
recurredcorr = correlation_matrix.Recurred
toberemoved = list(recurredcorr[(recurredcorr<0.1) & (recurredcorr>-0.1)].index)
X = X.drop(columns=toberemoved + ['M','N','T'])

Splitting data, getting 20% in test dataset.

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=47)

## KNN Classifier  
Training a series of KNN classifiers for different values of hyper parameters using GridSearchCV. Different values of number of neighbors and different kinds of distance metrics are considered. Then compare the accuracy to get the best model.

In [35]:
param_grid = {'n_neighbors' : list(range(1,25)), 
              'p' :[1,2,3]} # p parameter specifies the distance used, l1(manhattan), l2(euclidean), l3
knn = KNeighborsClassifier()
KNN_ = GridSearchCV(knn, param_grid, cv=3)
KNN_.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24],
                         'p': [1, 2, 3]})

In [196]:
scoresdf = pd.DataFrame(KNN_.cv_results_['params'])
scoresdf['mean_test_score'] = KNN_.cv_results_['mean_test_score']

alt.Chart(scoresdf).mark_point().encode(
    alt.Y('mean_test_score', scale=alt.Scale(domain=[0.90,0.95])),
    alt.X('n_neighbors'),
    alt.Color('p:N'),
    alt.Size('p:O'),
    tooltip = ['mean_test_score']
).interactive()

alt.Chart(...)

## Decision Tree Classifier  
Training a series of DT classifiers for different pruning alpha coefficient. Then compare the accuracy to get the best model.

In [139]:
from sklearn.tree import DecisionTreeClassifier

param_grid = {'min_samples_leaf' :list(range(1,6)),
              'ccp_alpha':np.linspace(0, 0.1, 10).tolist(),
              'random_state':[50]}
dt = DecisionTreeClassifier()
DT_ = GridSearchCV(dt, param_grid, cv=3)
DT_.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'ccp_alpha': [0.0, 0.011111111111111112,
                                       0.022222222222222223,
                                       0.03333333333333333,
                                       0.044444444444444446,
                                       0.05555555555555556, 0.06666666666666667,
                                       0.07777777777777778, 0.08888888888888889,
                                       0.1],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'random_state': [50]})

In [197]:
scoresdf = pd.DataFrame(DT_.cv_results_['params'])
scoresdf['mean_test_score'] = DT_.cv_results_['mean_test_score']

alt.Chart(scoresdf).mark_point().encode(
    alt.Y('mean_test_score', scale=alt.Scale(type='pow', exponent=10)),
    alt.X('ccp_alpha'),
    alt.Color('min_samples_leaf:N'),
    alt.Size('min_samples_leaf:O'),
    tooltip = ['mean_test_score']
).interactive()


alt.Chart(...)

## Random Forest Classifier  
Training a series of Random Forest classifiers for different values of hyper parameters (number of estimators, max_depth and min_samples_leaf). Then compare the accuracy to get the best model.

In [106]:
from sklearn.ensemble import RandomForestClassifier

param_grid = {'n_estimators': list(range(100, 500, 100)),
              'max_depth': [None, 2, 5, 10],#deeper trees can capture more details but are more prone to overfitting;
              'min_samples_leaf': [5, 8, 12], #the minimum number of samples in a leaf can reduce overfitting, indirectly limiting the depth of the trees
              'random_state':[50]}
rndf = RandomForestClassifier()
RndF_ = GridSearchCV(rndf, param_grid, cv=3)
RndF_.fit(X_train,y_train)


GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 2, 5, 10],
                         'min_samples_leaf': [5, 8, 12],
                         'n_estimators': [100, 200, 300, 400],
                         'random_state': [50]})

In [107]:
RndF_.best_params_, RndF_.best_score_

({'max_depth': None,
  'min_samples_leaf': 8,
  'n_estimators': 300,
  'random_state': 50},
 np.float64(0.945017182130584))

In [209]:
scoresdf = pd.DataFrame(RndF_.cv_results_['params'])
scoresdf['mean_test_score'] = RndF_.cv_results_['mean_test_score']

alt.Chart(scoresdf).mark_point().encode(
    alt.Y('mean_test_score', scale=alt.Scale(domain=[0.92, 0.96])),
    alt.X('n_estimators'),
    alt.Color('max_depth:N'),
    alt.Size('min_samples_leaf:O'),
    tooltip = ['mean_test_score']
).interactive()

alt.Chart(...)

## Best model  
The best model in each group

In [208]:
print('Best models parameters:')
print('KNN:', KNN_.best_params_)
print('DT:',DT_.best_params_)
print('RndF:', RndF_.best_params_)

Best models parameters:
KNN: {'n_neighbors': 3, 'p': 1}
DT: {'ccp_alpha': 0.011111111111111112, 'min_samples_leaf': 1, 'random_state': 50}
RndF: {'max_depth': None, 'min_samples_leaf': 8, 'n_estimators': 300, 'random_state': 50}


Comparing the three best models in respect to the accuracy on the test data.

In [216]:
print('Scores:')
print('KNN: ', KNN_.best_estimator_.score(X_test, y_test))
print('Decision Tree: ', DT_.best_estimator_.score(X_test, y_test))
print('Random Forest: ', RndF_.best_estimator_.score(X_test, y_test))

Scores:
KNN:  0.9178082191780822
Decision Tree:  0.9315068493150684
Random Forest:  0.9452054794520548


The random Forest model has the higest accuracy on the test dataset.  

Comparing the ROC curves to understand wich model has the best prediction power.

In [232]:
def roc_auc_df(model):
  y_prob = model.predict_proba(X_test)[:, 1]
  fpr, tpr, thresholds = roc_curve(y_test, y_prob)
  roc_auc = roc_auc_score(y_test, y_prob)
  roc_data = pd.DataFrame({
      'False Positive Rate': fpr,
      'True Positive Rate': tpr,
      'AUC': roc_auc,
      'Model': str(model).split('(')[0], #+ 'AUC:' +str(round(roc_auc,4)),
  })
  return roc_data

df = pd.concat([roc_auc_df(model.best_estimator_) for model in [KNN_, DT_, RndF_]])

In [233]:
alt.Chart(df).mark_line(color='blue').encode(
    x=alt.X('False Positive Rate', title='False Positive Rate'),
    y=alt.Y('True Positive Rate', title='True Positive Rate'),
    color=alt.Color('Model'),
    tooltip = ['Model', 'AUC']
).properties(
    width=400,
    height=400,
    title=f'ROC Curve'
).configure_legend(orient='right', labelLimit= 0, )

alt.Chart(...)

In [244]:
df[['Model', 'AUC']].drop_duplicates().sort_values('AUC', ascending=False)

,Model,AUC
0,RandomForestClassifier,0.984244
0,KNeighborsClassifier,0.951155
0,DecisionTreeClassifier,0.900210


The Random Forest model has the highest Area Under the Curve value, so it has the greatest prediction power.

## Exporting the model
The Random Forest model has been demstrated to be the best of the three. Now I export it in pkl format to use for prediction  in an application.

In [47]:
rndf_model = RndF_.best_estimator_.fit(X,y)
joblib.dump(rndf_model, 'rndf_model.pkl')

['rndf_model.pkl']

## Conclusions  
The analysis in this notebook demonstrates the process of preparing and modeling a dataset on thyroid risk using various machine learning algorithms. The Random Forest model, which was determined to be the best performing model, was fine-tuned using GridSearchCV. The Random Forest model achieved a high AUC score, indicating strong predictive performance. The model was subsequently saved for future use in prediction applications.